--- Day 12: Garden Groups ---
Why not search for the Chief Historian near the gardener and his massive farm? There's plenty of food, so The Historians grab something to eat while they search.

You're about to settle near a complex arrangement of garden plots when some Elves ask if you can lend a hand. They'd like to set up fences around each region of garden plots, but they can't figure out how much fence they need to order or how much it will cost. They hand you a map (your puzzle input) of the garden plots.

Each garden plot grows only a single type of plant and is indicated by a single letter on your map. When multiple garden plots are growing the same type of plant and are touching (horizontally or vertically), they form a region. For example:

AAAA
BBCD
BBCC
EEEC
This 4x4 arrangement includes garden plots growing five different types of plants (labeled A, B, C, D, and E), each grouped into their own region.

In order to accurately calculate the cost of the fence around a single region, you need to know that region's area and perimeter.

The area of a region is simply the number of garden plots the region contains. The above map's type A, B, and C plants are each in a region of area 4. The type E plants are in a region of area 3; the type D plants are in a region of area 1.

Each garden plot is a square and so has four sides. The perimeter of a region is the number of sides of garden plots in the region that do not touch another garden plot in the same region. The type A and C plants are each in a region with perimeter 10. The type B and E plants are each in a region with perimeter 8. The lone D plot forms its own region with perimeter 4.

Visually indicating the sides of plots in each region that contribute to the perimeter using - and |, the above map's regions' perimeters are measured as follows:

+-+-+-+-+
|A A A A|
+-+-+-+-+     +-+
              |D|
+-+-+   +-+   +-+
|B B|   |C|
+   +   + +-+
|B B|   |C C|
+-+-+   +-+ +
          |C|
+-+-+-+   +-+
|E E E|
+-+-+-+
Plants of the same type can appear in multiple separate regions, and regions can even appear within other regions. For example:

OOOOO
OXOXO
OOOOO
OXOXO
OOOOO
The above map contains five regions, one containing all of the O garden plots, and the other four each containing a single X plot.

The four X regions each have area 1 and perimeter 4. The region containing 21 type O plants is more complicated; in addition to its outer edge contributing a perimeter of 20, its boundary with each X region contributes an additional 4 to its perimeter, for a total perimeter of 36.

Due to "modern" business practices, the price of fence required for a region is found by multiplying that region's area by its perimeter. The total price of fencing all regions on a map is found by adding together the price of fence for every region on the map.

In the first example, region A has price 4 * 10 = 40, region B has price 4 * 8 = 32, region C has price 4 * 10 = 40, region D has price 1 * 4 = 4, and region E has price 3 * 8 = 24. So, the total price for the first example is 140.

In the second example, the region with all of the O plants has price 21 * 36 = 756, and each of the four smaller X regions has price 1 * 4 = 4, for a total price of 772 (756 + 4 + 4 + 4 + 4).

Here's a larger example:

RRRRIICCFF
RRRRIICCCF
VVRRRCCFFF
VVRCCCJFFF
VVVVCJJCFE
VVIVCCJJEE
VVIIICJJEE
MIIIIIJJEE
MIIISIJEEE
MMMISSJEEE
It contains:

A region of R plants with price 12 * 18 = 216.
A region of I plants with price 4 * 8 = 32.
A region of C plants with price 14 * 28 = 392.
A region of F plants with price 10 * 18 = 180.
A region of V plants with price 13 * 20 = 260.
A region of J plants with price 11 * 20 = 220.
A region of C plants with price 1 * 4 = 4.
A region of E plants with price 13 * 18 = 234.
A region of I plants with price 14 * 22 = 308.
A region of M plants with price 5 * 12 = 60.
A region of S plants with price 3 * 8 = 24.
So, it has a total price of 1930.

What is the total price of fencing all regions on your map?

To begin, get your puzzle input.

Answer: 
------------------------------------------------
How to find a "region"?
    1. find all plots with the same plant (letter) adjacent to each other
    2. it doesn't matter what type of plants. As long as they are not "together", they can be treated as different regions (a new index will be created regardless the type of plants)
    3. calculat each region's area and perimeter
        area = number of plots
        perimeter = (mumber of plots) * 4 - sum( number of each plot adjacent to other plots in the same region)
    4. Traverse the map (top left to botoom right):
        a. if the plot has alreday in a region, skip it to the next
        b. if the plot doesn't belong to a region, assign the plot to a region ID (letter&next number, e.g. A3)
            i. traverse all adjacent plots (breadth-first-search) that have the same letter (should not be not assigned yet)
            ii. record the number of plots as area, also record the adjacent plots for each plot in the region and sum them up to calculate the perimeter (=4*area - sum(adjacent plots for each plot))



In [89]:
def get_adjacent_records(matrix):
    def dfs(x, y, value, visited, cluster):
        if (x, y) in visited or not (0 <= x < len(matrix) and 0 <= y < len(matrix[0])) or matrix[x][y] != value:    # if already visited or not its neighbour node, return
            return
        visited.add((x, y))     # add to the visited set
        cluster.add((x, y))  # add to the cluster list
        for dx, dy in [(-1, 0), (1, 0), (0, -1), (0, 1)]:
            dfs(x + dx, y + dy, value, visited, cluster)    # recursive

    visited = set()     # quicker for later search
    clusters = []       # mutable list for updates

    for i in range(len(matrix)):
        for j in range(len(matrix[0])):
            if (i, j) not in visited:       # first node that has not visited
                cluster = set()                # reset the cluster list
                dfs(i, j, matrix[i][j], visited, cluster)
                if len(cluster) >= 1:       # allow one element cluster
                    clusters.append(cluster)

    return clusters



##*****************************
# Part I Test sample map
##*****************************
strMap = """RRRRIICCFF
RRRRIICCCF
VVRRRCCFFF
VVRCCCJFFF
VVVVCJJCFE
VVIVCCJJEE
VVIIICJJEE
MIIIIIJJEE
MIIISIJEEE
MMMISSJEEE"""


lstMap = []
for line in strMap.splitlines():
    lstMap.append(line)
#print(lstMap)

regions = get_adjacent_records(lstMap)
#print("Adjacent records with the same value:", regions)


TotalPrice = 0

## calculate perimeter for each region
## check for connections for each plot: perimeter = 4*area - sum of connections for each plot
for region in regions:
    print(region)
    count = 0
    for plot in region:
        (x,y) = plot
        for dx, dy in [(-1, 0), (1, 0), (0, -1), (0, 1)]:
            if (x + dx, y + dy) in region:
                count += 1

    perimeter = len(region)*4-count

    ## price for each region
    TotalPrice += perimeter*len(region)
    print(len(region), perimeter, perimeter*len(region))

## Total Price
print("Total Price=", TotalPrice)




{(0, 1), (2, 4), (1, 2), (0, 0), (1, 1), (0, 3), (2, 3), (0, 2), (2, 2), (1, 0), (3, 2), (1, 3)}
12 18 216
{(0, 5), (0, 4), (1, 4), (1, 5)}
4 8 32
{(4, 4), (0, 7), (5, 5), (3, 4), (6, 5), (5, 4), (1, 8), (0, 6), (1, 7), (3, 3), (2, 6), (1, 6), (2, 5), (3, 5)}
14 28 392
{(3, 8), (2, 7), (3, 7), (0, 9), (2, 9), (3, 9), (4, 8), (0, 8), (1, 9), (2, 8)}
10 18 180
{(4, 0), (2, 1), (4, 3), (3, 1), (6, 1), (2, 0), (5, 1), (4, 2), (3, 0), (5, 0), (6, 0), (5, 3), (4, 1)}
13 20 260
{(7, 7), (9, 6), (4, 6), (5, 7), (6, 7), (4, 5), (7, 6), (5, 6), (8, 6), (3, 6), (6, 6)}
11 20 220
{(4, 7)}
1 4 4
{(8, 8), (9, 7), (9, 9), (4, 9), (5, 8), (6, 8), (8, 7), (7, 9), (8, 9), (9, 8), (5, 9), (6, 9), (7, 8)}
13 18 234
{(7, 4), (6, 2), (7, 1), (9, 3), (8, 1), (6, 4), (7, 3), (8, 3), (7, 2), (8, 2), (7, 5), (6, 3), (8, 5), (5, 2)}
14 22 308
{(9, 0), (7, 0), (9, 2), (8, 0), (9, 1)}
5 12 60
{(9, 5), (8, 4), (9, 4)}
3 8 24
Total Price= 1930


In [83]:
##*****************************
# Part I def functions
##*****************************
def get_adjacent_records(matrix):
    def dfs(x, y, value, visited, cluster):
        if (x, y) in visited or not (0 <= x < len(matrix) and 0 <= y < len(matrix[0])) or matrix[x][y] != value:    # if already visited or not its neighbour node, return
            return
        visited.add((x, y))     # add to the visited set
        cluster.add((x, y))  # add to the cluster list
        for dx, dy in [(-1, 0), (1, 0), (0, -1), (0, 1)]:
            dfs(x + dx, y + dy, value, visited, cluster)    # recursive

    visited = set()     # quicker for later search
    clusters = []       # mutable list for updates

    for i in range(len(matrix)):
        for j in range(len(matrix[0])):
            if (i, j) not in visited:       # first node that has not visited
                cluster = set()                # reset the cluster list
                dfs(i, j, matrix[i][j], visited, cluster)
                if len(cluster) >= 1:       # allow one element cluster
                    clusters.append(cluster)

    return clusters



##***********************************************
## *****   Part I Main Program Start Here   *****
##***********************************************

lstMap = []
##-------------------------------------------------
## load data file
##-------------------------------------------------
with open('D:\Work\AdventOfCode\Data\Day 12 Data.txt','r') as f:
    for line in f:
        lstMap.append(line.replace("\n",""))


regions = get_adjacent_records(lstMap)
#print("Adjacent records with the same value:", regions)


TotalPrice = 0

## calculate perimeter for each region
## check for connections for each plot: perimeter = 4*area - sum of connections for each plot
for region in regions:
    #print(region)
    count = 0
    for plot in region:
        (x,y) = plot
        for dx, dy in [(-1, 0), (1, 0), (0, -1), (0, 1)]:
            if (x + dx, y + dy) in region:
                count += 1

    perimeter = len(region)*4-count

    ## price for each region
    TotalPrice += perimeter*len(region)
    #print(len(region), perimeter, perimeter*len(region))

## Total Price
print("Total Price=", TotalPrice)




Total Price= 1494342


--- Part Two ---
Fortunately, the Elves are trying to order so much fence that they qualify for a bulk discount!

Under the bulk discount, instead of using the perimeter to calculate the price, you need to use the number of sides each region has. Each straight section of fence counts as a side, regardless of how long it is.

Consider this example again:

AAAA
BBCD
BBCC
EEEC
The region containing type A plants has 4 sides, as does each of the regions containing plants of type B, D, and E. However, the more complex region containing the plants of type C has 8 sides!

Using the new method of calculating the per-region price by multiplying the region's area by its number of sides, regions A through E have prices 16, 16, 32, 4, and 12, respectively, for a total price of 80.

The second example above (full of type X and O plants) would have a total price of 436. (21 plots * 20 sides + 1 plot * 4 sides * 4 regions)

Here's a map that includes an E-shaped region full of type E plants:

EEEEE
EXXXX
EEEEE
EXXXX
EEEEE

The E-shaped region has an area of 17 and 12 sides for a price of 204. Including the two regions full of type X plants, this map has a total price of 236.

This map has a total price of 368:

AAAAAA
AAABBA
AAABBA
ABBAAA
ABBAAA
AAAAAA

It includes two regions full of type B plants (each with 4 sides) and a single region full of type A plants (with 4 sides on the outside and 8 more sides on the inside, a total of 12 sides). Be especially careful when counting the fence around regions like the one full of type A plants; in particular, each section of fence has an in-side and an out-side, so the fence does not connect across the middle of the region (where the two B regions touch diagonally). (The Elves would have used the Möbius Fencing Company instead, but their contract terms were too one-sided.)

The larger example from before now has the following updated prices:

A region of R plants with price 12 * 10 = 120.
A region of I plants with price 4 * 4 = 16.
A region of C plants with price 14 * 22 = 308.
A region of F plants with price 10 * 12 = 120.
A region of V plants with price 13 * 10 = 130.
A region of J plants with price 11 * 12 = 132.
A region of C plants with price 1 * 4 = 4.
A region of E plants with price 13 * 8 = 104.
A region of I plants with price 14 * 16 = 224.
A region of M plants with price 5 * 6 = 30.
A region of S plants with price 3 * 6 = 18.
Adding these together produces its new total price of 1206.

What is the new total price of fencing all regions on your map?

Answer: 
-----------------------------------------------------
How to calculate "sizes" of a region:  
    1. follow the perimeter calculation (4*area - sum of number of adjacent plots )  
    2. for each plot  
        a. if both left and current plot don't have plot going up,   deduct 1 from perimeter  
        b. if both left and current plot don't have plot going down, deduct 1 from perimeter  
        c. if both up and current plot don't have plot going left,   deduct 1 from perimeter  
        d. if both up and current plot don't have plot going right,  deduct 1 from perimeter  

In [94]:
##*****************************
# Part II def functions
##*****************************
def get_adjacent_records(matrix):
    def dfs(x, y, value, visited, cluster):
        if (x, y) in visited or not (0 <= x < len(matrix) and 0 <= y < len(matrix[0])) or matrix[x][y] != value:    # if already visited or not its neighbour node, return
            return
        visited.add((x, y))     # add to the visited set
        cluster.add((x, y))  # add to the cluster list
        for dx, dy in [(-1, 0), (1, 0), (0, -1), (0, 1)]:
            dfs(x + dx, y + dy, value, visited, cluster)    # recursive

    visited = set()     # quicker for later search
    clusters = []       # mutable list for updates

    for i in range(len(matrix)):
        for j in range(len(matrix[0])):
            if (i, j) not in visited:       # first node that has not visited
                cluster = set()                # reset the cluster list
                dfs(i, j, matrix[i][j], visited, cluster)
                if len(cluster) >= 1:       # allow one element cluster
                    clusters.append(cluster)

    return clusters



##***********************************************
## *****   Part II Main Program Start Here   *****
##***********************************************

lstMap = []
##-------------------------------------------------
## load data file
##-------------------------------------------------
with open('D:\Work\AdventOfCode\Data\Day 12 Data.txt','r') as f:
    for line in f:
        lstMap.append(line.replace("\n",""))


regions = get_adjacent_records(lstMap)
#print("Adjacent records with the same value:", regions)



TotalPrice = 0

## calculate sides for each region
##
for region in regions:
    #print(region)
    discount = 0
    for plot in region:
        (x,y) = plot
        ## consider the connections to its adjacent plots
        for dx, dy in [(-1, 0), (1, 0), (0, -1), (0, 1)]:
            if (x + dx, y + dy) in region:
                discount += 1

        ## consider the continuous plots for the side of the current plot belongs to
        ## (x-1,y-1), (x-1,y), (x-1,y+1)
        ## (x  ,y-1), (x  ,y), (x  ,y+1)
        ## (x+1,y-1), (x+1,y), (x+1,y+1)
        if (x,y-1) in region and (x-1,y-1) not in region and (x-1,y) not in region:     ## if has left side plot, both have no facing up plot
            discount += 1
        if (x,y-1) in region and (x+1,y-1) not in region and (x+1,y) not in region:     ## if has left side plot, both have no facing down plot
            discount += 1
        if (x-1,y) in region and (x-1,y-1) not in region and (x,y-1) not in region:     ## if has up side plot, both have no facing left plot
            discount += 1
        if (x-1,y) in region and (x-1,y+1) not in region and (x,y+1) not in region:     ## if has up side plot, both have no facing right plot
            discount += 1


    perimeter = len(region)*4-discount

    ## price for each region
    TotalPrice += perimeter*len(region)
    print(len(region), perimeter, perimeter*len(region))

## Total Price
print("Total Price=", TotalPrice)




88 38 3344
23 14 322
108 48 5184
14 16 224
59 28 1652
1 4 4
5 6 30
68 38 2584
5 8 40
11 10 110
69 34 2346
5 6 30
145 66 9570
38 16 608
200 74 14800
1 4 4
95 10 950
34 22 748
72 38 2736
15 18 270
1 4 4
91 44 4004
40 26 1040
49 30 1470
166 84 13944
1 4 4
1 4 4
1 4 4
1 4 4
2 4 8
56 38 2128
3 6 18
91 46 4186
121 54 6534
58 32 1856
1 4 4
35 26 910
1 4 4
59 30 1770
3 4 12
58 32 1856
1 4 4
122 38 4636
6 10 60
1 4 4
2 4 8
41 22 902
2 4 8
1 4 4
135 84 11340
1 4 4
17 14 238
1 4 4
128 50 6400
7 10 70
1 4 4
191 24 4584
98 38 3724
1 4 4
26 20 520
36 28 1008
104 50 5200
1 4 4
1 4 4
1 4 4
1 4 4
15 10 150
1 4 4
42 34 1428
64 30 1920
32 18 576
135 58 7830
4 8 32
6 8 48
142 52 7384
1 4 4
3 6 18
49 34 1666
1 4 4
1 4 4
2 4 8
1 4 4
1 4 4
86 28 2408
24 16 384
1 4 4
43 34 1462
1 4 4
2 4 8
119 56 6664
1 4 4
2 4 8
9 14 126
28 20 560
21 16 336
1 4 4
11 12 132
136 48 6528
133 60 7980
48 32 1536
46 32 1472
4 4 16
7 10 70
1 4 4
2 4 8
16 16 256
50 22 1100
51 26 1326
1 4 4
3 6 18
255 80 20400
1 4 4
41 28 1148
10 12 